In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

* 7월 17일 제헌절 : 2008년부터 
* 4월 5일 식목일 : 2006년부터 


In [2]:
# 공휴일 API로 1996~2034 공휴일 데이타 받아오기 
import requests
from bs4 import BeautifulSoup
import time
import datetime

def get_request_query(url, operation, params, serviceKey):
    import urllib.parse as urlparse
    params = urlparse.urlencode(params)
    request_query = url + '/' + operation + '?' + params + '&' + 'serviceKey' + '=' + serviceKey
    return request_query

# url 불러오기
URL = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService'
OPERATION = 'getHoliDeInfo' 

# 서비스키 입력
SERVICEKEY = "G3p6Cjw2s%2B4zCKLHU6KmxazwUogf7seHbU9kTM%2Bl1FUxOCJpNureZdSTXl8w%2BadLHNzSgzCSOkMDDrTStTeD6g%3D%3D"
def year_month(year, month):
    solYear  = str(year)  
    if month >= 10:
        solMonth = str(month)    
    else:
        solMonth = '0'+str(month)
    PARAMS = {'solYear':solYear, 'solMonth':solMonth}


    request_query = get_request_query(URL, OPERATION, PARAMS, SERVICEKEY)
    print('request_query:', request_query)
    response = requests.get(url=request_query)
    print('status_code:' + str(response.status_code))
    
    bsObj = BeautifulSoup(response.text, 'lxml')
    datename = bsObj.find_all("datename")
    datename = bsObj.find_all("datename")
    datename = list(map(lambda x : x.get_text(),datename))
    locdate = bsObj.find_all("locdate")
    locdate = list(map(lambda x : x.get_text(),locdate))
    df = pd.DataFrame(zip(locdate,datename))
    return(df)

df = pd.DataFrame()
for i in  range(1996,2034):
    for j in range(1,13):
        df=df.append(year_month(i,j) , ignore_index=True)

df.rename(columns = {0: "date", 1:"holiday"},inplace=True)

request_query: http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo?solYear=1996&solMonth=01&serviceKey=G3p6Cjw2s%2B4zCKLHU6KmxazwUogf7seHbU9kTM%2Bl1FUxOCJpNureZdSTXl8w%2BadLHNzSgzCSOkMDDrTStTeD6g%3D%3D
status_code:200
request_query: http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo?solYear=1996&solMonth=02&serviceKey=G3p6Cjw2s%2B4zCKLHU6KmxazwUogf7seHbU9kTM%2Bl1FUxOCJpNureZdSTXl8w%2BadLHNzSgzCSOkMDDrTStTeD6g%3D%3D
status_code:200
request_query: http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo?solYear=1996&solMonth=03&serviceKey=G3p6Cjw2s%2B4zCKLHU6KmxazwUogf7seHbU9kTM%2Bl1FUxOCJpNureZdSTXl8w%2BadLHNzSgzCSOkMDDrTStTeD6g%3D%3D
status_code:200
request_query: http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo?solYear=1996&solMonth=04&serviceKey=G3p6Cjw2s%2B4zCKLHU6KmxazwUogf7seHbU9kTM%2Bl1FUxOCJpNureZdSTXl8w%2BadLHNzSgzCSOkMDDrTStTeD6g%3D%3D
status_code:200
request_query: http://ap

In [3]:
df['date'] = df['date'].str[0:4]+'-'+ df['date'].str[4:6]+'-'+df['date'].str[6:8] # API날짜 연월일 형식 변경
#df = df[~df.date.str.contains("07-17",na=False)].reset_index(drop=True) # 제헌절이 API데이타에 있어 삭제 
df = df.drop_duplicates(['date'],keep='last') #2017/10/03 개천절이자 추석. 이런날 제거 
# df = df[:106] #train_public 날짜인 2019-03-31 전데이타만 자름 
df

,date,holiday
0,2004-01-01,신정
1,2004-01-21,설날
2,2004-01-22,설날
3,2004-01-23,설날
4,2004-03-01,삼일절
...,...,...
358,2024-09-17,추석
359,2024-09-18,추석
360,2024-10-03,개천절
361,2024-10-09,한글날


In [4]:
df1 = df.assign(holi=np.nan)  # holi 빈칸열 만들기 (공휴일이름은 0,1로 바꾸기 위해 )

In [5]:
df1['holi'] = df1['holi'].fillna(1) #모두 1로 채우기 
##df1 = df1[['date','holi']] # date랑 holi열만 가져오기 
df1

,date,holiday,holi
0,2004-01-01,신정,1.0
1,2004-01-21,설날,1.0
2,2004-01-22,설날,1.0
3,2004-01-23,설날,1.0
4,2004-03-01,삼일절,1.0
...,...,...,...
358,2024-09-17,추석,1.0
359,2024-09-18,추석,1.0
360,2024-10-03,개천절,1.0
361,2024-10-09,한글날,1.0


In [6]:
# 1996-2034까지 날짜 뽑아서 주말은 1로 만들기 
date = pd.DataFrame(pd.date_range(start='2004-01-01', end='2024-12-31',  freq='D', name='date'))
date['date'] = pd.to_datetime(date['date']) 
date['dayname'] = date['date'].dt.weekday # 월~일을 0~6으로 맵핑 

In [7]:
date['dayname']= date['dayname'].replace({1:0,2:0, 3:0, 4:0, 5:1,6:1}) # 토,일만 1로 하고 나머지는 0으로 바꾸기 
date['date']= date['date'].astype(str) 
date

,date,dayname
0,2004-01-01,0
1,2004-01-02,0
2,2004-01-03,1
3,2004-01-04,1
4,2004-01-05,0
...,...,...
7666,2024-12-27,0
7667,2024-12-28,1
7668,2024-12-29,1
7669,2024-12-30,0


In [8]:
df2 = pd.merge(date,df1,how='outer',on='date')  # 공휴일과 주말 데이타 merge 
df2 = df2.fillna(0)  # 공휴일을 뺀 나머지 날 0으로 바꾸기 
df_2004_2024 = df2[['date','holi']]
df_2004_2024

,date,holi
0,2004-01-01,1.0
1,2004-01-02,0.0
2,2004-01-03,0.0
3,2004-01-04,0.0
4,2004-01-05,0.0
...,...,...
7666,2024-12-27,0.0
7667,2024-12-28,0.0
7668,2024-12-29,0.0
7669,2024-12-30,0.0


# 수동작업 1996-2003

In [9]:
from datetime import datetime  
from datetime import timedelta 

In [10]:
#1996
d = datetime(1996,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(1996,1,1)
,datetime(1996,1,2)
,datetime(1996,2,18)
,datetime(1996,2,19)
,datetime(1996,2,20)
,datetime(1996,3,1)
,datetime(1996,4,5)
,datetime(1996,5,5)
,datetime(1996,5,24)
,datetime(1996,6,6)
,datetime(1996,7,17)
,datetime(1996,8,15)
,datetime(1996,9,26)
,datetime(1996,9,27)
,datetime(1996,9,28)
,datetime(1996,10,3)
,datetime(1996,12,25)]


holiday = set(holiday)
df_1996=pd.DataFrame(data={'date':days,'holi':1})
df_1996['holi']= (df_1996['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_1996['date']=df_1996['date'].astype(str)
print(df_1996.head(3))
print(df_1996.info())

         date  holi
0  1996-01-01     1
1  1996-01-02     1
2  1996-01-03     0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [11]:
#1997
d = datetime(1997,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(1997,1,1)
,datetime(1997,1,2)
,datetime(1997,2,7)
,datetime(1997,2,8)
,datetime(1997,2,9)
,datetime(1997,3,1)
,datetime(1997,4,5)
,datetime(1997,5,5)
,datetime(1997,5,14)
,datetime(1997,6,6)
,datetime(1997,7,17)
,datetime(1997,8,15)
,datetime(1997,9,15)
,datetime(1997,9,16)
,datetime(1997,9,17)
,datetime(1997,10,3)
,datetime(1997,12,18)
,datetime(1997,12,25)]

holiday = set(holiday)
df_1997=pd.DataFrame(data={'date':days,'holi':1})
df_1997['holi']= (df_1997['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_1997['date']=df_1997['date'].astype(str)
print(df_1997.head(3))
print(df_1997.info())

         date  holi
0  1997-01-01     1
1  1997-01-02     1
2  1997-01-03     0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [12]:
#1998
d = datetime(1998,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(1998,1,1)
,datetime(1998,1,2)
,datetime(1998,1,27)
,datetime(1998,1,28)
,datetime(1998,1,29)
,datetime(1998,3,1)
,datetime(1998,4,5)
,datetime(1998,5,3)
,datetime(1998,5,5)
,datetime(1998,6,6)
,datetime(1998,7,17)
,datetime(1998,8,15)
,datetime(1998,10,3)
,datetime(1998,10,4)
,datetime(1998,10,5)
,datetime(1998,10,6)
,datetime(1998,12,25)]

holiday = set(holiday)
df_1998=pd.DataFrame(data={'date':days,'holi':1})
df_1998['holi']= (df_1998['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_1998['date']=df_1998['date'].astype(str)
print(df_1998.head(3))
print(df_1998.info())

         date  holi
0  1998-01-01     1
1  1998-01-02     1
2  1998-01-03     1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [13]:
#1999
d = datetime(1999,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(1999,1,1)
,datetime(1999,2,15)
,datetime(1999,2,16)
,datetime(1999,2,17)
,datetime(1999,3,1)
,datetime(1999,4,5)
,datetime(1999,5,5)
,datetime(1999,5,22)
,datetime(1999,6,6)
,datetime(1999,7,17)
,datetime(1999,8,15)
,datetime(1999,9,23)
,datetime(1999,9,24)
,datetime(1999,9,25)
,datetime(1999,10,3)
,datetime(1999,12,25)]

holiday = set(holiday)
df_1999=pd.DataFrame(data={'date':days,'holi':1})
df_1999['holi']= (df_1999['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_1999['date']=df_1999['date'].astype(str)
print(df_1999.head(3))
print(df_1999.info())

         date  holi
0  1999-01-01     1
1  1999-01-02     1
2  1999-01-03     1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [14]:
#2000
d = datetime(2000,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(2000,1,1)
,datetime(2000,2,4)
,datetime(2000,2,5)
,datetime(2000,2,6)
,datetime(2000,3,1)
,datetime(2000,4,5)
,datetime(2000,5,5)
,datetime(2000,5,11)
,datetime(2000,6,6)
,datetime(2000,7,17)
,datetime(2000,8,15)
,datetime(2000,9,11)
,datetime(2000,9,12)
,datetime(2000,9,13)
,datetime(2000,10,3)
,datetime(2000,12,25)]

holiday = set(holiday)
df_2000=pd.DataFrame(data={'date':days,'holi':1})
df_2000['holi']= (df_2000['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_2000['date']=df_2000['date'].astype(str)
print(df_2000.head(3))
print(df_2000.info())

         date  holi
0  2000-01-01     1
1  2000-01-02     1
2  2000-01-03     0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [15]:
#2001
d = datetime(2001,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(2001,1,1)
,datetime(2001,1,23)
,datetime(2001,1,24)
,datetime(2001,1,25)
,datetime(2001,3,1)
,datetime(2001,4,5)
,datetime(2001,5,1)
,datetime(2001,5,5)
,datetime(2001,6,6)
,datetime(2001,7,17)
,datetime(2001,8,15)
,datetime(2001,9,30)
,datetime(2001,10,1)
,datetime(2001,10,2)
,datetime(2001,10,3)
,datetime(2001,12,25)]

holiday = set(holiday)
df_2001=pd.DataFrame(data={'date':days,'holi':1})
df_2001['holi']= (df_2001['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_2001['date']=df_2001['date'].astype(str)
print(df_2001.head(3))
print(df_2001.info())

         date  holi
0  2001-01-01     1
1  2001-01-02     0
2  2001-01-03     0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [16]:
#2002
d = datetime(2002,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(2002,1,1)
,datetime(2002,2,11)
,datetime(2002,2,12)
,datetime(2002,2,13)
,datetime(2002,3,1)
,datetime(2002,4,5)
,datetime(2002,5,5)
,datetime(2002,5,19)
,datetime(2002,6,6)
,datetime(2002,7,17)
,datetime(2002,8,15)
,datetime(2002,9,20)
,datetime(2002,9,21)
,datetime(2002,9,22)
,datetime(2002,10,3)
,datetime(2002,12,19)
,datetime(2002,12,25)]

holiday = set(holiday)
df_2002=pd.DataFrame(data={'date':days,'holi':1})
df_2002['holi']= (df_2002['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_2002['date']=df_2002['date'].astype(str)
print(df_2002.head(3))
print(df_2002.info())

         date  holi
0  2002-01-01     1
1  2002-01-02     0
2  2002-01-03     0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [17]:
#2003
d = datetime(2003,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(2003,1,1)
,datetime(2003,1,31)
,datetime(2003,2,1)
,datetime(2003,2,2)
,datetime(2003,3,1)
,datetime(2003,4,5)
,datetime(2003,5,5)
,datetime(2003,5,8)
,datetime(2003,6,6)
,datetime(2003,7,17)
,datetime(2003,8,15)
,datetime(2003,9,10)
,datetime(2003,9,11)
,datetime(2003,9,12)
,datetime(2003,10,3)
,datetime(2003,12,25)]

holiday = set(holiday)
df_2003=pd.DataFrame(data={'date':days,'holi':1})
df_2003['holi']= (df_2003['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_2003['date']=df_2003['date'].astype(str)
print(df_2003.head(3))
print(df_2003.info())

         date  holi
0  2003-01-01     1
1  2003-01-02     0
2  2003-01-03     0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [18]:
df_2003

,date,holi
0,2003-01-01,1
1,2003-01-02,0
2,2003-01-03,0
3,2003-01-04,1
4,2003-01-05,1
...,...,...
360,2003-12-27,1
361,2003-12-28,1
362,2003-12-29,0
363,2003-12-30,0


In [19]:
holiday_1996_2024 = pd.concat([df_1996,df_1997,df_1998,df_1999,df_2000,df_2001,df_2002,df_2003,df_2004_2024],axis=0)

In [20]:
holiday_1996_2024.reset_index(drop=True, inplace=True)
holiday_1996_2024

,date,holi
0,1996-01-01,1.0
1,1996-01-02,1.0
2,1996-01-03,0.0
3,1996-01-04,0.0
4,1996-01-05,0.0
...,...,...
10586,2024-12-27,0.0
10587,2024-12-28,0.0
10588,2024-12-29,0.0
10589,2024-12-30,0.0


In [21]:
#2025
d = datetime(2025,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(2025,1,1)
,datetime(2025,1,28)
,datetime(2025,1,29)
,datetime(2025,1,30)
,datetime(2025,3,1)
,datetime(2025,3,3)
,datetime(2025,5,5)
,datetime(2025,5,6)
,datetime(2025,6,6)
,datetime(2025,8,15)
,datetime(2025,10,3)
,datetime(2025,10,5)
,datetime(2025,10,6)
,datetime(2025,10,7)
,datetime(2025,10,8)
,datetime(2025,10,9)
,datetime(2025,12,25)]

holiday = set(holiday)
df_2025=pd.DataFrame(data={'date':days,'holi':1})
df_2025['holi']= (df_2025['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_2025['date']=df_2025['date'].astype(str)
print(df_2025.head(3))
print(df_2025.info())

         date  holi
0  2025-01-01     1
1  2025-01-02     0
2  2025-01-03     0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [22]:
#2026
d = datetime(2026,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(2026,1,1)
,datetime(2026,2,16)
,datetime(2026,2,17)
,datetime(2026,2,18)
,datetime(2026,3,1)
,datetime(2026,3,2)
,datetime(2026,5,5)
,datetime(2026,5,24)
,datetime(2026,6,6)
,datetime(2026,8,15)
,datetime(2026,8,17)
,datetime(2026,9,24)
,datetime(2026,9,25)
,datetime(2026,9,26)
,datetime(2026,10,3)
,datetime(2026,10,5)
,datetime(2026,10,9)
,datetime(2026,12,25)]

holiday = set(holiday)
df_2026=pd.DataFrame(data={'date':days,'holi':1})
df_2026['holi']= (df_2026['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_2026['date']=df_2026['date'].astype(str)
print(df_2026.head(3))
print(df_2026.info())

         date  holi
0  2026-01-01     1
1  2026-01-02     0
2  2026-01-03     1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [23]:
#2027
d = datetime(2027,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(2027,1,1)
,datetime(2027,2,6)
,datetime(2027,2,7)
,datetime(2027,2,8)
,datetime(2027,2,9)
,datetime(2027,3,1)
,datetime(2027,3,3)
,datetime(2027,5,5)
,datetime(2027,5,13)
,datetime(2027,6,6)
,datetime(2027,8,15)
,datetime(2027,8,16)
,datetime(2027,9,14)
,datetime(2027,9,15)
,datetime(2027,9,16)
,datetime(2027,10,3)
,datetime(2027,10,4)
,datetime(2027,10,9)
,datetime(2027,10,11)
,datetime(2027,12,25)]

holiday = set(holiday)
df_2027=pd.DataFrame(data={'date':days,'holi':1})
df_2027['holi']= (df_2027['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_2027['date']=df_2027['date'].astype(str)
print(df_2027.head(3))
print(df_2027.info())

         date  holi
0  2027-01-01     1
1  2027-01-02     1
2  2027-01-03     1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [24]:
#2028
d = datetime(2028,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(2028,1,1)
,datetime(2028,1,26)
,datetime(2028,1,27)
,datetime(2028,1,28)
,datetime(2028,3,1)
,datetime(2028,5,2)
,datetime(2028,5,5)
,datetime(2028,6,6)
,datetime(2028,8,15)
,datetime(2028,10,2)
,datetime(2028,10,3)
,datetime(2028,10,4)
,datetime(2028,10,5)
,datetime(2028,10,9)
,datetime(2028,12,25)]

holiday = set(holiday)
df_2028=pd.DataFrame(data={'date':days,'holi':1})
df_2028['holi']= (df_2028['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_2028['date']=df_2028['date'].astype(str)
print(df_2028.head(3))
print(df_2028.info())

         date  holi
0  2028-01-01     1
1  2028-01-02     1
2  2028-01-03     0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [25]:
#2029
d = datetime(2029,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(2029,1,1)
,datetime(2029,2,12)
,datetime(2029,2,13)
,datetime(2029,2,14)
,datetime(2029,3,1)
,datetime(2029,5,5)
,datetime(2029,5,7)
,datetime(2029,5,20)
,datetime(2029,6,6)
,datetime(2029,8,15)
,datetime(2029,9,21)
,datetime(2029,9,22)
,datetime(2029,9,23)
,datetime(2029,9,24)
,datetime(2029,10,3)
,datetime(2029,10,9)
,datetime(2029,12,25)]

holiday = set(holiday)
df_2029=pd.DataFrame(data={'date':days,'holi':1})
df_2029['holi']= (df_2029['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_2029['date']=df_2029['date'].astype(str)
print(df_2029.head(3))
print(df_2029.info())

         date  holi
0  2029-01-01     1
1  2029-01-02     0
2  2029-01-03     0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [26]:
#2030
d = datetime(2030,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(2030,1,1)
,datetime(2030,2,2)
,datetime(2030,2,3)
,datetime(2030,2,4)
,datetime(2030,2,5)
,datetime(2030,3,1)
,datetime(2030,5,5)
,datetime(2030,5,6)
,datetime(2030,5,9)
,datetime(2030,6,6)
,datetime(2030,8,15)
,datetime(2030,9,11)
,datetime(2030,9,12)
,datetime(2030,9,13)
,datetime(2030,10,3)
,datetime(2030,10,9)
,datetime(2030,12,25)]

holiday = set(holiday)
df_2030=pd.DataFrame(data={'date':days,'holi':1})
df_2030['holi']= (df_2030['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_2030['date']=df_2030['date'].astype(str)
print(df_2030.head(3))
print(df_2030.info())

         date  holi
0  2030-01-01     1
1  2030-01-02     0
2  2030-01-03     0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [27]:
#2031
d = datetime(2031,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(2031,1,1)
,datetime(2031,1,22)
,datetime(2031,1,23)
,datetime(2031,1,24)
,datetime(2031,3,1)
,datetime(2031,3,3)
,datetime(2031,5,5)
,datetime(2031,5,28)
,datetime(2031,6,6)
,datetime(2031,8,15)
,datetime(2031,9,30)
,datetime(2031,10,1)
,datetime(2031,10,2)
,datetime(2031,10,3)
,datetime(2031,10,9)
,datetime(2031,12,25)]

holiday = set(holiday)
df_2031=pd.DataFrame(data={'date':days,'holi':1})
df_2031['holi']= (df_2031['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_2031['date']=df_2031['date'].astype(str)
print(df_2031.head(3))
print(df_2031.info())

         date  holi
0  2031-01-01     1
1  2031-01-02     0
2  2031-01-03     0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [28]:
#2032
d = datetime(2032,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(2032,1,1)
,datetime(2032,2,10)
,datetime(2032,2,11)
,datetime(2032,2,12)
,datetime(2032,3,1)
,datetime(2032,5,5)
,datetime(2032,5,16)
,datetime(2032,6,6)
,datetime(2032,8,15)
,datetime(2032,8,16)
,datetime(2032,9,18)
,datetime(2032,9,20)
,datetime(2032,9,21)
,datetime(2032,10,3)
,datetime(2032,10,4)
,datetime(2032,10,9)
,datetime(2032,10,11)
,datetime(2032,12,25)]

holiday = set(holiday)
df_2032=pd.DataFrame(data={'date':days,'holi':1})
df_2032['holi']= (df_2032['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_2032['date']=df_2032['date'].astype(str)
print(df_2032.head(3))
print(df_2032.info())

         date  holi
0  2032-01-01     1
1  2032-01-02     0
2  2032-01-03     1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [29]:
#2033
d = datetime(2033,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(2033,1,1)
,datetime(2033,1,30)
,datetime(2033,1,31)
,datetime(2033,2,2)
,datetime(2033,3,1)
,datetime(2033,5,5)
,datetime(2033,5,6)
,datetime(2033,6,6)
,datetime(2033,8,15)
,datetime(2033,9,7)
,datetime(2033,9,8)
,datetime(2033,9,9)
,datetime(2033,10,3)
,datetime(2033,10,9)
,datetime(2033,10,10)
,datetime(2033,12,25)]

holiday = set(holiday)
df_2033=pd.DataFrame(data={'date':days,'holi':1})
df_2033['holi']= (df_2033['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_2033['date']=df_2033['date'].astype(str)
print(df_2033.head(3))
print(df_2033.info())

         date  holi
0  2033-01-01     1
1  2033-01-02     1
2  2033-01-03     0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [30]:
#2034
d = datetime(2034,1,1)
days = [d + timedelta(days=i) for i in range(0,365) ]
holiday = [d  for d in days if d.weekday() in [5,6] ]
#기본 주말 
holiday += [datetime(2034,1,1)
,datetime(2034,2,18)
,datetime(2034,2,19)
,datetime(2034,2,20)
,datetime(2034,2,21)
,datetime(2034,3,1)
,datetime(2034,5,5)
,datetime(2034,5,25)
,datetime(2034,6,6)
,datetime(2034,8,15)
,datetime(2034,9,26)
,datetime(2034,9,27)
,datetime(2034,9,28)
,datetime(2034,10,3)
,datetime(2034,10,9)
,datetime(2034,12,25)]

holiday = set(holiday)
df_2034=pd.DataFrame(data={'date':days,'holi':1})
df_2034['holi']= (df_2034['date'].isin(holiday)).apply(lambda x : 1 if x else 0)
df_2034['date']=df_2034['date'].astype(str)
print(df_2034.head(3))
print(df_2034.info())

         date  holi
0  2034-01-01     1
1  2034-01-02     0
2  2034-01-03     0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    365 non-null    object
 1   holi    365 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
None


In [31]:
holiday_1996_2034 = pd.concat([holiday_1996_2024,df_2025,df_2026,df_2027,df_2028,df_2029,df_2030,df_2031,df_2032,df_2033,df_2034],axis=0)

In [ ]:
holiday_1996_2034.reset_index(drop=True, inplace=True)
holiday_1996_2034.to_csv('~/Desktop/finalCopy/0_data/newdata/holiday_1996_2034.csv',index=False) 

In [7]:
# holiday rate 만들기 
holiday = pd.read_csv('~/Desktop/finalCopy/0_data/newdata/holiday_1996_2034.csv')
holiday['yearmonth'] = holiday['date'].str.split('-').str[0] + holiday['date'].str.split('-').str[1]
holiday['year']= holiday['date'].str.split('-').str[0]
holiday['month']= holiday['date'].str.split('-').str[1]

holiday['date']= holiday['date'].str.split('-').str[2]
holiday_max = pd.DataFrame(holiday.groupby(['year','month'])['date'].max()).reset_index()
holiday1 = pd.DataFrame(holiday.groupby('yearmonth')['holi'].sum()).reset_index()
holiday_max['yearmonth'] = holiday_max['year']+ holiday_max['month']
holi = pd.merge(holiday_max,holiday1, how='inner', on='yearmonth')
holi['holirate'] = holi['holi']/holi['date'].astype(int)
holi.to_csv('~/Desktop/finalCopy/0_data/newdata/holirate_2034.csv', index=False) 

In [ ]:
## new df_00 불러와서 holirate 채우기 
#new_df = pd.read_csv('~/Desktop/finalCopy/0_data/newdata/new_df_00_rp3.csv')
#new_df['holirate'] = holi['holirate']
#new_df.to_csv('~/Desktop/finalCopy/0_data/newdata/new_df_00_rp3.csv', index=False) 
